# Grupo 3 del proyecto de data mining - Analisis de datos

***Authors : H. Kanza, N.Matte, J. Escrihuela, A. Ortiz, ...***

In [16]:
# Imports
import requests
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile
import os
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from io import StringIO


# I. Import data from the website

First, I download the HTML page where I can find the URLs of every monthly file I want to analyse.

In [17]:
# URL of the file I want to download
url_page = "https://www.omie.es/en/file-access-list?parents%5B0%5D=/&parents%5B1%5D=Day-ahead%20Market&parents%5B2%5D=3.%20Curves&dir=Monthly%20files%20with%20aggregate%20supply%20and%20demand%20curves%20of%20Day-ahead%20market%20including%20bid%20units&realdir=curva_pbc_uof"

# Path where I want to save the file
path = "data/webpage_omie.html"

# GET request to try to access URL data
response = requests.get(url_page)

# I make sure the request succeeded
if response.status_code == 200:
    # ... so I write the data in a local file, in the specified path
    with open(path, "wb") as f:
        f.write(response.content)
    print(f"File saved at {path}")
else:
    print(f"Request error code: {response.status_code}")


File saved at data/webpage_omie.html


Now, with the HTML, I need to extract the list containing the URLs of all curves files.

In [18]:
# I read the HTML file
with open(path, "r", encoding="utf-8") as file:
    html_content = file.read()

    # Pattern with characters before and after
    pattern = r".*curva_pbc_uof_*."

    # Find occurrences that match the pattern
    matches = re.findall(pattern, html_content)

    if matches:
        print(len(matches), "occurences found on the webpage.")
    else:
        print("No occurence found.")

76 occurences found on the webpage.


In [19]:
# URL de la página con la lista de archivos
url_page = "https://www.omie.es/en/file-access-list?parents%5B0%5D=/&parents%5B1%5D=Intraday%20Auction%20Market&parents%5B2%5D=3.%20Curves&dir=Monthly%20files%20with%20aggregate%20supply%20and%20demnand%20curves%20of%20intraday%20auction%20market%20including%20bid%20units&realdir=curva_pibc_uof"

# Hacer una solicitud GET para obtener el contenido de la página
response = requests.get(url_page)

if response.status_code == 200:
    # Parsear el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Buscar todos los enlaces en la página que coincidan con el patrón de nombre del archivo
    links = soup.find_all('a', href=True)
    file_names = [re.search(r'curva_pibc_uof_\d{6}.zip', link['href']).group(0) for link in links if re.search(r'curva_pibc_uof_\d{6}.zip', link['href'])]

    if file_names:
        # Ordenar los nombres de archivos por fecha de más reciente a más antiguo
        file_names.sort(reverse=True)

        # Obtener el nombre del archivo más reciente
        latest_file_name = file_names[0]

        # Construir la URL completa del archivo más reciente encontrado
        latest_file_url = f"https://www.omie.es/en/file-download?parents%5B0%5D=curva_pibc_uof&filename={latest_file_name}"
        # Nombre del archivo a guardar en el disco
        file_name = f"data/{latest_file_name}"

        # Hacer una solicitud GET para descargar el archivo ZIP
        response = requests.get(latest_file_url)

        if response.status_code == 200:
            # Guardar el archivo ZIP en la ruta especificada
            os.makedirs(os.path.dirname(file_name), exist_ok=True)
            with open(file_name, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Descarga completa: {file_name}")
        else:
            print(f"Error al descargar el archivo: {response.status_code}")
    else:
        print("No se encontraron archivos que coincidan con el patrón especificado.")
else:
    print(f"Error en la solicitud a la página: {response.status_code}")

Descarga completa: data/curva_pibc_uof_202402.zip


In [20]:
directorio_destino = 'descomprimido/'
# Extraer el archivo zip
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(directorio_destino)

In [21]:
# Obtener la lista de archivos descomprimidos
archivos_descomprimidos = os.listdir(directorio_destino)

In [22]:
# Cambiar el nombre de los archivos .1 a .csv
for archivo in archivos_descomprimidos:
    if archivo.endswith('.1'):
        # Construir las rutas de origen y destino
        ruta_origen = os.path.join(directorio_destino, archivo)
        ruta_destino = os.path.join(directorio_destino, archivo[:-2] + '.csv')
        # Renombrar el archivo
        os.rename(ruta_origen, ruta_destino)

In [23]:
# Obtener la lista de archivos CSV en el directorio
archivos_descomprimidos = [f for f in os.listdir(directorio_destino) if f.endswith('.csv')]

# Inicializar un DataFrame vacío para almacenar los datos combinados
data_frame_combinado = pd.DataFrame()


In [64]:
# Iterar sobre los archivos CSV y leerlos en el DataFrame
for archivo in archivos_descomprimidos:
    ruta_archivo = os.path.join(directorio_destino, archivo)
    # Leer el archivo saltando las dos primeras filas y usando la tercera fila como encabezado
    datos_archivo = pd.read_csv(ruta_archivo, skiprows=2,delimiter=';')
    # Concatenar los datos al DataFrame combinado
    data_frame_combinado = pd.concat([data_frame_combinado, datos_archivo], ignore_index=True)


In [65]:
data_frame_combinado.head()

,Hora,Fecha,Pais,Unidad,Tipo Oferta,Energ�a Compra/Venta,Precio Compra/Venta,Ofertada (O)/Casada (C),Unnamed: 8
0,1.0,01/02/2024,MI,WMVD101,C,"0,2","1.002,00",O,NaN
1,1.0,01/02/2024,MI,WMVD147,C,"0,1","1.002,00",O,NaN
2,1.0,01/02/2024,MI,IBEVD15,C,"2,0","801,00",O,NaN
3,1.0,01/02/2024,MI,IBEVD14,C,"2,0","801,00",O,NaN
4,1.0,01/02/2024,MI,NXVD219,C,"49,5","700,00",O,NaN


In [ ]:
# (JAIME MARTIN) In this Section, We are going to clean and prepare Data obtained from various
# sources related to the OMIE SPOT Electricity
#1 Handly Missing Values
#2 Removing Outliers
#3 Data Normalization 

